## YOUR FIRST LAB
### Your first Frontier LLM Project

Let's build a useful LLM solution - in a matter of minutes.

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!


In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

### Connecting to OpenAI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  

If you'd like to use free Ollama instead, please see the README section "Free Alternative to Paid APIs", and if you're not sure how to do this, there's a full solution in the solutions folder (day1_with_ollama.ipynb).

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.


### Let's make a quick call to a Frontier model to get started, as a preview!

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to have you here. How can I assist you today?


### OK onwards with our first project

In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
# print(ed.text)

Home - Edward Donner


### Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [10]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [11]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, let me think... it's almost like a math riddle! The answer is 4. Shocking, I know!


### And now let's build useful messages for GPT-4o-mini, using a function

In [12]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

### Time to bring it together - the API for OpenAI is very simple!

In [14]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nThe website is primarily a personal platform for Ed Donner, who is a code enthusiast and co-founder and CTO of Nebula.io. The platform focuses on advancements in AI, specifically LLMs (Large Language Models), and their application in talent discovery and management. Ed also shares his interests, including DJing and electronic music production.\n\n## Key Highlights:\n- **Professional Background**: Ed is the co-founder of Nebula.io, which helps companies recruit and manage talent using AI. He was previously the founder of the AI startup untapt, which was acquired in 2021.\n- **Innovative Work**: Nebula.io employs proprietary LLMs and has developed a patented matching model, receiving positive attention and press coverage.\n\n## Recent Announcements:\n- **May 28, 2025**: Introduction of courses aimed at helping individuals become LLM experts and leaders.\n- **May 18, 2025**: Announcement of a 2025 AI Executive Briefing.\n- **April 21, 2025**: Laun

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website serves as a platform for sharing his interests and expertise in coding and large language models (LLMs). He is the co-founder and CTO of Nebula.io, a company focused on using AI to help individuals discover their potential and effectively manage talent. Previously, he was the founder and CEO of an AI startup called untapt, which was acquired in 2021.

## Recent Announcements and Courses
- **May 28, 2025**: Announcement about connecting courses to help people become experts and leaders in LLMs.
- **May 18, 2025**: Information regarding a 2025 AI Executive Briefing.
- **April 21, 2025**: Introduction of "The Complete Agentic AI Engineering Course".
- **January 23, 2025**: Details on an LLM Workshop featuring hands-on resources with agents. 

Overall, the site is a resource for those interested in AI and LLMs, featuring educational content and personal insights from Edward.

### Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [18]:
display_summary("https://cnn.com")

# CNN Website Summary

CNN's website serves as a hub for breaking news, current events, and in-depth analysis across various topics including politics, business, health, entertainment, science, and global conflicts. Key highlights include:

## Recent News:
- **Trump’s Domestic Policy Bill**: The U.S. Congress has passed a significant domestic policy bill, marking a legislative achievement for President Trump amid contentious debates.
- **Celebrity Deaths**: Notable figures such as actor Michael Madsen and football star Diogo Jota have recently passed away, prompting tributes from fans and peers.
- **Global Conflicts**: Ongoing issues include intensified strikes in Gaza and significant developments in the Ukraine-Russia war, including the death of a high-ranking Russian officer.

## Other Noteworthy Stories:
- A serious environmental crisis with wildfires disrupting tourism in Greece.
- A unique AI-related story where a couple achieved pregnancy after a long struggle.
- **Cultural and Lifestyle Trends**: Articles discuss various contemporary issues such as upcycling fashion, the best pizza in America, and the impact of a four-day workweek based on recent trials.

The website also covers an array of sports news, health analyses, technology innovations, and international relations, striving to provide comprehensive coverage of relevant global issues.

In [19]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

Anthropic is focused on developing advanced AI technology, particularly through their flagship model, Claude. The website provides information on various aspects of Claude, including different model versions such as Claude Opus 4 and Sonnet 4, which highlight their capabilities in coding and AI agent functionalities.

## Key Features:
- **Claude Models**: Claude Opus 4 is touted as the most intelligent model available, with enhancements for coding and carrying out complex tasks.
- **API and Development**: The site encourages collaboration by offering APIs for developers to build custom AI-powered applications.
- **Safety and Ethics**: Anthropic emphasizes the importance of responsible AI development and safety, outlining their commitments and policies aimed at ensuring human benefit in technology advancements.

## Recent Announcements:
- **ISO 42001 Certification**: A recent announcement highlights Anthropic's achievement in obtaining ISO 42001 certification, which may indicate their commitment to standardized best practices in AI development.

## Educational Resources:
- The **Anthropic Academy** is available for those interested in learning how to work with Claude, offering resources for both beginners and more experienced developers.

Overall, Anthropic is dedicated to advancing AI technology responsibly while prioritizing safety and societal impact.